In [8]:
##########CREATE MODEL INCEPTIONResNet V1#######################################
from keras.layers import Input, merge, Dropout, Dense, Lambda, Flatten, Activation
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import MaxPooling2D, Convolution2D, AveragePooling2D
from keras.models import Model

from keras import backend as K

import warnings
warnings.filterwarnings('ignore')

"""
Implementation of Inception-Residual Network v1 [Inception Network v4 Paper](http://arxiv.org/pdf/1602.07261v1.pdf) in Keras.

Some additional details:
[1] Each of the A, B and C blocks have a 'scale_residual' parameter.
    The scale residual parameter is according to the paper. It is however turned OFF by default.

    Simply setting 'scale=True' in the create_inception_resnet_v1() method will add scaling.
"""


def inception_resnet_stem(input):
    if K.image_dim_ordering() == "th":
        channel_axis = 1
    else:
        channel_axis = -1

    # Input Shape is 299 x 299 x 3 (tf) or 3 x 299 x 299 (th)
    c = Convolution2D(32, 3, 3, activation='relu', subsample=(2, 2))(input)
    c = Convolution2D(32, 3, 3, activation='relu', )(c)
    c = Convolution2D(64, 3, 3, activation='relu', )(c)
    c = MaxPooling2D((3, 3), strides=(2, 2))(c)
    c = Convolution2D(80, 1, 1, activation='relu', border_mode='same')(c)
    c = Convolution2D(192, 3, 3, activation='relu')(c)
    c = Convolution2D(256, 3, 3, activation='relu', subsample=(2,2), border_mode='same')(c)
    b = BatchNormalization(axis=channel_axis)(c)
    b = Activation('relu')(b)
    return b

def inception_resnet_A(input, scale_residual=True):
    if K.image_dim_ordering() == "th":
        channel_axis = 1
    else:
        channel_axis = -1

    # Input is relu activation
    init = input

    ir1 = Convolution2D(32, 1, 1, activation='relu', border_mode='same')(input)

    ir2 = Convolution2D(32, 1, 1, activation='relu', border_mode='same')(input)
    ir2 = Convolution2D(32, 3, 3, activation='relu', border_mode='same')(ir2)

    ir3 = Convolution2D(32, 1, 1, activation='relu', border_mode='same')(input)
    ir3 = Convolution2D(32, 3, 3, activation='relu', border_mode='same')(ir3)
    ir3 = Convolution2D(32, 3, 3, activation='relu', border_mode='same')(ir3)

    ir_merge = merge([ir1, ir2, ir3], concat_axis=channel_axis, mode='concat')

    ir_conv = Convolution2D(256, 1, 1, activation='linear', border_mode='same')(ir_merge)
    if scale_residual: ir_conv = Lambda(lambda x: x * 0.1)(ir_conv)

    out = merge([init, ir_conv], mode='sum')
    out = BatchNormalization(axis=channel_axis)(out)
    out = Activation("relu")(out)
    return out

def inception_resnet_B(input, scale_residual=True):
    if K.image_dim_ordering() == "th":
        channel_axis = 1
    else:
        channel_axis = -1

    # Input is relu activation
    init = input

    ir1 = Convolution2D(128, 1, 1, activation='relu', border_mode='same')(input)

    ir2 = Convolution2D(128, 1, 1, activation='relu', border_mode='same')(input)
    ir2 = Convolution2D(128, 1, 7, activation='relu', border_mode='same')(ir2)
    ir2 = Convolution2D(128, 7, 1, activation='relu', border_mode='same')(ir2)

    ir_merge = merge([ir1, ir2], mode='concat', concat_axis=channel_axis)

    ir_conv = Convolution2D(896, 1, 1, activation='linear', border_mode='same')(ir_merge)
    if scale_residual: ir_conv = Lambda(lambda x: x * 0.1)(ir_conv)

    out = merge([init, ir_conv], mode='sum')
    out = BatchNormalization(axis=channel_axis)(out)
    out = Activation("relu")(out)
    return out

def inception_resnet_C(input, scale_residual=True):
    if K.image_dim_ordering() == "th":
        channel_axis = 1
    else:
        channel_axis = -1

    # Input is relu activation
    init = input

    ir1 = Convolution2D(128, 1, 1, activation='relu', border_mode='same')(input)

    ir2 = Convolution2D(192, 1, 1, activation='relu', border_mode='same')(input)
    ir2 = Convolution2D(192, 1, 3, activation='relu', border_mode='same')(ir2)
    ir2 = Convolution2D(192, 3, 1, activation='relu', border_mode='same')(ir2)

    ir_merge = merge([ir1, ir2], mode='concat', concat_axis=channel_axis)

    ir_conv = Convolution2D(1792, 1, 1, activation='linear', border_mode='same')(ir_merge)
    if scale_residual: ir_conv = Lambda(lambda x: x * 0.1)(ir_conv)

    out = merge([init, ir_conv], mode='sum')
    out = BatchNormalization(axis=channel_axis)(out)
    out = Activation("relu")(out)
    return out

def reduction_A(input, k=192, l=224, m=256, n=384):
    if K.image_dim_ordering() == "th":
        channel_axis = 1
    else:
        channel_axis = -1

    r1 = MaxPooling2D((3,3), strides=(2,2))(input)

    r2 = Convolution2D(n, 3, 3, activation='relu', subsample=(2,2))(input)

    r3 = Convolution2D(k, 1, 1, activation='relu', border_mode='same')(input)
    r3 = Convolution2D(l, 3, 3, activation='relu', border_mode='same')(r3)
    r3 = Convolution2D(m, 3, 3, activation='relu', subsample=(2,2))(r3)

    m = merge([r1, r2, r3], mode='concat', concat_axis=channel_axis)
    m = BatchNormalization(axis=channel_axis)(m)
    m = Activation('relu')(m)
    return m


def reduction_resnet_B(input):
    if K.image_dim_ordering() == "th":
        channel_axis = 1
    else:
        channel_axis = -1

    r1 = MaxPooling2D((3,3), strides=(2,2), border_mode='valid')(input)

    r2 = Convolution2D(256, 1, 1, activation='relu', border_mode='same')(input)
    r2 = Convolution2D(384, 3, 3, activation='relu', subsample=(2,2))(r2)

    r3 = Convolution2D(256, 1, 1, activation='relu', border_mode='same')(input)
    r3 = Convolution2D(256, 3, 3, activation='relu', subsample=(2, 2))(r3)

    r4 = Convolution2D(256, 1, 1, activation='relu', border_mode='same')(input)
    r4 = Convolution2D(256, 3, 3, activation='relu', border_mode='same')(r4)
    r4 = Convolution2D(256, 3, 3, activation='relu', subsample=(2, 2))(r4)

    m = merge([r1, r2, r3, r4], concat_axis=channel_axis, mode='concat')
    m = BatchNormalization(axis=channel_axis)(m)
    m = Activation('relu')(m)
    return m

def create_inception_resnet_v1(nb_classes=1001, scale=True):
    '''
    Creates a inception resnet v1 network

    :param nb_classes: number of classes.txt
    :param scale: flag to add scaling of activations
    :return: Keras Model with 1 input (299x299x3) input shape and 2 outputs (final_output, auxiliary_output)
    '''

    if K.image_dim_ordering() == 'th':
        init = Input((3, 299, 299))
    else:
        init = Input((299, 299, 3))

    # Input Shape is 299 x 299 x 3 (tf) or 3 x 299 x 299 (th)
    x = inception_resnet_stem(init)

    # 5 x Inception Resnet A
    for i in range(5):
        x = inception_resnet_A(x, scale_residual=scale)

    # Reduction A - From Inception v4
    x = reduction_A(x, k=192, l=192, m=256, n=384)

    # 10 x Inception Resnet B
    for i in range(10):
        x = inception_resnet_B(x, scale_residual=scale)

    # Auxiliary tower
    aux_out = AveragePooling2D((5, 5), strides=(3, 3))(x)
    aux_out = Convolution2D(128, 1, 1, border_mode='same', activation='relu')(aux_out)
    aux_out = Convolution2D(768, 5, 5, activation='relu')(aux_out)
    aux_out = Flatten()(aux_out)
    aux_out = Dense(nb_classes, activation='softmax')(aux_out)

    # Reduction Resnet B
    x = reduction_resnet_B(x)

    # 5 x Inception Resnet C
    for i in range(5):
        x = inception_resnet_C(x, scale_residual=scale)

    # Average Pooling
    x = AveragePooling2D((8,8))(x)

    # Dropout
    x = Dropout(0.8)(x)
    x = Flatten()(x)

    # Output
    out = Dense(output_dim=nb_classes, activation='softmax')(x)
    model = Model(init, output=[out, aux_out], name='Inception-Resnet-v1')
    #model = Model(init, output=[x, aux_out], name='Inception-Resnet-v1')
    return model

if __name__ == "__main__":
    
    import  matplotlib.pyplot  as  plt

    inception_resnet_v1 = create_inception_resnet_v1()
    #plt.savefig(os.path.join(PATH_TO_DF,  "Inception ResNet-v1.png"),  bbox_inches='tight') 
    #plot(inception_resnet_v1, to_file="Inception ResNet-v1.png", show_shapes=True)
    inception_resnet_v1.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 299, 299, 3)  0                                            
__________________________________________________________________________________________________
conv2d_130 (Conv2D)             (None, 149, 149, 32) 896         input_2[0][0]                    
__________________________________________________________________________________________________
conv2d_131 (Conv2D)             (None, 147, 147, 32) 9248        conv2d_130[0][0]                 
__________________________________________________________________________________________________
conv2d_132 (Conv2D)             (None, 145, 145, 64) 18496       conv2d_131[0][0]                 
__________________________________________________________________________________________________
max_poolin

In [9]:
import os, shutil
from os import walk, getcwd
# Retorna todos los archivos de un directorio dado
def ls(ruta):  
    return next(walk(ruta))[2]

height_image = 299
width_image = 299
channels_image = 3
nb_clases = 4
batch_size = 2
class_mode = 'categorical'
nb_train = 4000        # 1000 x 4 clases
nb_validation = 1200   #  300 x 4 clases
nb_test = 1200

base_dir = 'balanced_dataset'
train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'validation')
test_dir = os.path.join(base_dir, 'test')

In [10]:
#Extracción de características del modelo Preentrenado y nuestro dataset
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
from keras.utils import to_categorical

#Tamaño de salida de la última capa convoluciónal del modelo preentrenado
#lo vemos en el conv_base.summary()
out_x = 2
out_y = 2
conv_len = 1001

datagen = ImageDataGenerator(rescale = 1./255)

def extract_features(directory, sample_count):
    features = np.zeros(shape=(sample_count, out_x, out_x, conv_len))
    labels = to_categorical(np.zeros(shape=(sample_count)),nb_clases) #INDICAR NUMERO DE CLASES
    generator = datagen.flow_from_directory(
        directory,
        target_size = (height_image, width_image),
        batch_size = batch_size,
        #classes = 4,
        class_mode = 'categorical')
    i = 0
    for inputs_batch, labels_batch in generator:
        features_batch = inception_resnet_v1.predict(inputs_batch) ##Asociamos al modelo preentrenado
        features[i * batch_size : (i + 1) * batch_size] = features_batch
        labels[i * batch_size : (i + 1) * batch_size] = labels_batch
        i += 1
        if i * batch_size >= sample_count:
            break
    return features, labels

#Train: 1000 muestras x clase, con 4 clases, 1000 x 4 = 4000
train_features, train_labels = extract_features(train_dir, nb_train)
#validation 500 muestras x clase con 4 clases, 300 x 4 = 1200
validation_features, validation_labels = extract_features(validation_dir, nb_validation) 
#test 500 muestras x clase con 4 clases, 300 x 4 = 1200
test_features, test_labels = extract_features(test_dir, nb_test)


Found 4000 images belonging to 4 classes.
Found 1200 images belonging to 4 classes.
Found 1200 images belonging to 4 classes.


In [5]:
#Las características extraídas están como muetras en forma (muestra, 4, 4, 512).-->(muestras, out_x, out_y, conv_len)
#Debemos aplanarlas (muestras, 8192) para alimentar un clasificador densamente conectado 4x4x512 = 8192 si entrada 150x150
#Debemos aplanarlas (muestras, 25088) para alimentar un clasificador densamente conectado 7x7x512 = 25088 si entrada 224x224


train_features = np.reshape(train_features, (nb_train, out_x * out_y * conv_len))
validation_features = np.reshape(validation_features, (nb_validation, out_x * out_y * conv_len))
test_features = np.reshape(test_features, (nb_test, out_x * out_y * conv_len))
print(train_features.shape)
print(validation_features.shape)
print(test_features.shape)

(4000, 4004)
(1200, 4004)
(1200, 4004)


In [6]:
#Definimos un clasificador densamente conectado capacitado con los datos y etiquetas obtenidas antes
#Aplicamos capa de abandono Dropout y función de activación "sigmoid" al final
from keras import layers
from keras import models
from keras import optimizers

top_model = models.Sequential()
top_model.add(layers.Dense(256, activation = 'relu', input_dim = out_x * out_y * conv_len))
top_model.add(layers.Dropout(0.5))
top_model.add(layers.Dense(nb_clases, activation = 'sigmoid'))  #INDICAR NUMERO DE CLASES

top_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_3 (Dense)              (None, 256)               1025280   
_________________________________________________________________
dropout_2 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 4)                 1028      
Total params: 1,026,308
Trainable params: 1,026,308
Non-trainable params: 0
_________________________________________________________________


In [7]:
#Para la compilación usaremos el optimizador RMSprop ya que termina la red con una sola
#unidad sigmoidea, cómo función de pérdida se utiliza la entropia cruzada binaria

top_model.compile(loss = 'categorical_crossentropy', 
              optimizer = optimizers.RMSprop(lr=2e-5), 
              metrics = ['acc'])

epochs = 80

history = top_model.fit(
    train_features,
    train_labels,  
    batch_size = batch_size,
    epochs = epochs, 
    verbose = 1, 
    validation_data = (validation_features, validation_labels))


Train on 4000 samples, validate on 1200 samples
Epoch 1/80
4000/4000 [==============================] - 5s 1ms/step - loss: 1.3864 - acc: 0.2382 - val_loss: 1.3863 - val_acc: 0.2500
Epoch 2/80
4000/4000 [==============================] - 5s 1ms/step - loss: 1.3863 - acc: 0.2513 - val_loss: 1.3863 - val_acc: 0.2500
Epoch 3/80
4000/4000 [==============================] - 5s 1ms/step - loss: 1.3864 - acc: 0.2447 - val_loss: 1.3863 - val_acc: 0.2500
Epoch 4/80
4000/4000 [==============================] - 5s 1ms/step - loss: 1.3863 - acc: 0.2467 - val_loss: 1.3863 - val_acc: 0.2500
Epoch 5/80
4000/4000 [==============================] - 5s 1ms/step - loss: 1.3864 - acc: 0.2505 - val_loss: 1.3863 - val_acc: 0.2500
Epoch 6/80
4000/4000 [==============================] - 5s 1ms/step - loss: 1.3866 - acc: 0.2370 - val_loss: 1.3863 - val_acc: 0.2500
Epoch 7/80
4000/4000 [==============================] - 5s 1ms/step - loss: 1.3862 - acc: 0.2575 - val_loss: 1.3864 - val_acc: 0.2500
Epoch 8/80
400